In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import pandas as pd
import json
import csv
import pickle
import numpy as np
        
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
counter = 0
core_name = "dc_cubes_historic"
predictionColumn = "cpuusage_ps"


In [3]:
def deleteSolrCore(core_name, deleteEverything=True):  # löscht immer den ganzen core
    url = "http://localhost:8983/solr/admin/cores?action=UNLOAD&core="+core_name

    if (deleteEverything):
        url += "&deleteInstanceDir=true"
    requests.get(url)
    print(core_name, " core deleted")

In [4]:
def pushData(row):
    global core_name
    global counter
    global allMetrics
    global predictionColumn
    # defining the api-endpoint
    url = "http://localhost:8983/solr/"+core_name+"/update/json/docs"
    # data to be sent to api
    data = {
                "timestamp": str(row["timestamp"]),
                "cluster": row["cluster"],
                "dc": row["dc"],
                "perm": -1, #row["perm"],
                "instanz": row["instanz"],
                "verfahren": "-1",# row["verfahren"],
                "service": "-1" ,#row["verfahren"],
                "response": -1 ,#row["response"],
                "count": row[predictionColumn],
                "minv":  -1,#row["minv"],
                "maxv":  -1,#row["maxv"],
                "avg": -1, #row["avg"],
                "var": -1 ,#row["var"],
                "dev_upp": -1, #row["dev_upp"],
                "dev_low": -1, #row["dev_low"],
                "perc90": -1 ,#row["perc90"],
                "perc95": -1 ,#row["perc95"],
                "perc99": -1 ,#row["perc99.9"],
                "sum": -1, #row["sum"],
                "sum_of_squares": -1, #row["sum_of_squares"],
                "server": "PBZ0%dE00_PERM02_S0%s_OSB" % (row["cluster"], row["instanz"]), #row["server"]
            }
    
    for metric in allMetrics:
        data[metric] = row[metric]
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    session.get(url)
    headers = {'Content-type': 'application/json'}
    # sending post request
    session.post(url=url, data=json.dumps(data), headers=headers)
    counter += 1
    if (counter % 1000 == 0):
        print("Commiting... counter:", counter)
        requests.get("http://localhost:8983/solr/"+core_name+"/update?commit=true")

In [5]:
def createSolrCore(core_name):
    url = "http://localhost:8983/solr/admin/cores?action=CREATE&name=" + \
        core_name+"&configSet=_default"
    requests.post(url=url)
    print(core_name, " created")



In [6]:
def initSchema(core_name, allMetrics):
    url = "http://localhost:8983/solr/"+core_name+"/schema"
    headers = {'Content-type': 'application/json'}
    rowsDict = {
        "timestamp": "pdate", "host": "string", "cluster": "pint", "dc": "pint", "perm": "pint", "instanz": "string", "verfahren": "string",
        "service": "string", "response": "pint", "count": "pfloat", "minv": "pint", "maxv": "pint", "avg": "pfloat", "var": "pfloat",
        "dev_upp": "pfloat", "dev_low": "pfloat", "perc90": "pfloat", "perc95": "pfloat", "perc99": "pfloat", "sum": "pint",
        "sum_of_squares": "pint", "server": "string"}

    for name in rowsDict:
        data = {
            "add-field": {"stored": "true", "docValues": "true", "indexed": "false", "multiValued": "false", "name": name, "type": rowsDict[name]}
        }
        requests.post(url=url, data=json.dumps(data), headers=headers)
        
    for metric in allMetrics:
        if metric not in rowsDict:
            data = {
                "add-field": {"stored": "true", "docValues": "true", "indexed": "false", "multiValued": "false", "name": metric, "type": "pfloat"}
            }
            requests.post(url=url, data=json.dumps(data), headers=headers)
    
    
    print(core_name, " schema inited")


In [7]:
def deleteCoreDocuments(core_name):
    url = "http://localhost:8983/solr/"+core_name + \
        "/update?commitWithin=1000&overwrite=true&wt=json"
    headers = {'Content-type': 'application/json'}
    data = {'delete': {'query': '*:*'}}
    requests.post(url=url, data=json.dumps(data), headers=headers)
    print("deleted old documents from "+core_name+" core")

In [8]:
def getData(core_name):
    # from 1 Month, there are about 90k entries. 2 clusters * 2 dcs * 8 instances * (4 weeks * 7 days * 24 hours * 4 (15min intervall))
    url = 'http://localhost:8983/solr/'+core_name+'/select?q=*:*&sort=timestamp%20asc&rows=100000'
    response = requests.get(url).json()['response']
    response
    return response['docs']

In [9]:
# get existing solr cores
url = "http://localhost:8983/solr/admin/cores?action=STATUS"
response = requests.get(url).json()
activeCores = response['status'].keys()

In [10]:
if core_name in activeCores:
    print(core_name + " already exists")
    # delete old data/predictions
    deleteCoreDocuments(core_name)
# else forecast core doesn't exist
else:
    print(core_name + " doesn't exist")
    # create an new forecast solr core
    createSolrCore(core_name)
    # init schema
    initSchema(core_name)

dc_cubes_historic already exists
deleted old documents from dc_cubes_historic core


In [11]:
# Read data from pickle file
with open("./data_pblm1.pkl", "rb") as pickleFile:
    pblm1 = pickle.load(pickleFile)
pblm1 = pblm1.reset_index()

In [12]:
# Read data from pickle file
with open("./data_pblm2.pkl", "rb") as pickleFile:
    pblm2 = pickle.load(pickleFile)
pblm2 = pblm2.reset_index()

In [13]:
pblm2.timestamp.max()

Timestamp('2020-01-23 09:30:00')

In [14]:
# missing 2020-01-23 09:30:00
copyRow = pblm2[pblm2.timestamp == pblm2.timestamp.max()].copy()

In [15]:
copyRow.timestamp = pd.Timestamp("2020-01-23 09:45:00")
copyRow

,index,timestamp,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,...,pxdwngrd25_ps,dictionarymiss_pct,cpuusage_ps,pxdwngrd50_ps,avg_tot_cpu_usage_ps,max_tot_cpu_usage_ps,dayOfWeek,isWeekend,hour,minute
2787,2787,2020-01-23 09:45:00,0.002,10240,20479.998,21.732,320,4928.0,14336,1213.082,...,0,0.0,0.449,0,3.005974,6.708903,3,0,9,30


In [16]:
pblm2 = pblm2.append(copyRow, ignore_index=True)

In [17]:
allMetrics = pblm2.columns

In [18]:
allMetrics = allMetrics.drop("timestamp")

In [19]:
pblm2[allMetrics] = pblm2[allMetrics].apply(pd.to_numeric, errors="coerce") # cast all to numeric because values arent getting pushed correctly to solr otherwise

In [20]:
pblm1[allMetrics] = pblm1[allMetrics].apply(pd.to_numeric, errors="coerce")

In [21]:
initSchema(core_name, allMetrics)

dc_cubes_historic  schema inited


In [22]:
# There are two dc's: 0 and 1
# dc 0 has clusters 6 and 8
# dc 1 has clusters 5 and 7
# Every Cluster has all 8 instances

def pushDataForAllInstances(df):
    instances = ["1","2","3","4","5","6","7","8" ]
    for instanz in instances:
        df["instanz"] = instanz
        df.apply(pushData, axis=1)

In [23]:
pblm2["dc"] = 0 # this will be pblm_2

In [24]:
pblm2["cluster"] = 6

In [25]:
pushDataForAllInstances(pblm2)

Commiting... counter: 1000
Commiting... counter: 2000
Commiting... counter: 3000
Commiting... counter: 4000
Commiting... counter: 5000
Commiting... counter: 6000
Commiting... counter: 7000
Commiting... counter: 8000
Commiting... counter: 9000
Commiting... counter: 10000
Commiting... counter: 11000
Commiting... counter: 12000
Commiting... counter: 13000
Commiting... counter: 14000
Commiting... counter: 15000
Commiting... counter: 16000
Commiting... counter: 17000
Commiting... counter: 18000
Commiting... counter: 19000
Commiting... counter: 20000
Commiting... counter: 21000
Commiting... counter: 22000


In [26]:
pblm2["cluster"] = 8

In [27]:
pushDataForAllInstances(pblm2)

Commiting... counter: 23000
Commiting... counter: 24000
Commiting... counter: 25000
Commiting... counter: 26000
Commiting... counter: 27000
Commiting... counter: 28000
Commiting... counter: 29000
Commiting... counter: 30000
Commiting... counter: 31000
Commiting... counter: 32000
Commiting... counter: 33000
Commiting... counter: 34000
Commiting... counter: 35000
Commiting... counter: 36000
Commiting... counter: 37000
Commiting... counter: 38000
Commiting... counter: 39000
Commiting... counter: 40000
Commiting... counter: 41000
Commiting... counter: 42000
Commiting... counter: 43000
Commiting... counter: 44000


In [28]:
pblm1["dc"] = 1

In [29]:
pblm1["cluster"] = 5

In [30]:
pushDataForAllInstances(pblm1)

Commiting... counter: 45000
Commiting... counter: 46000
Commiting... counter: 47000
Commiting... counter: 48000
Commiting... counter: 49000
Commiting... counter: 50000
Commiting... counter: 51000
Commiting... counter: 52000
Commiting... counter: 53000
Commiting... counter: 54000
Commiting... counter: 55000
Commiting... counter: 56000
Commiting... counter: 57000
Commiting... counter: 58000
Commiting... counter: 59000
Commiting... counter: 60000
Commiting... counter: 61000
Commiting... counter: 62000
Commiting... counter: 63000
Commiting... counter: 64000
Commiting... counter: 65000
Commiting... counter: 66000


In [31]:
pblm1["cluster"] = 7

In [32]:
pushDataForAllInstances(pblm1)

Commiting... counter: 67000
Commiting... counter: 68000
Commiting... counter: 69000
Commiting... counter: 70000
Commiting... counter: 71000
Commiting... counter: 72000
Commiting... counter: 73000
Commiting... counter: 74000
Commiting... counter: 75000
Commiting... counter: 76000
Commiting... counter: 77000
Commiting... counter: 78000
Commiting... counter: 79000
Commiting... counter: 80000
Commiting... counter: 81000
Commiting... counter: 82000
Commiting... counter: 83000
Commiting... counter: 84000
Commiting... counter: 85000
Commiting... counter: 86000
Commiting... counter: 87000
Commiting... counter: 88000
Commiting... counter: 89000


In [33]:
requests.get("http://localhost:8983/solr/"+core_name+"/update?commit=true")

<Response [200]>

In [34]:
#*______________________________________*#

In [35]:
forecast_core_name = "dc_cubes_forecast"

In [36]:
historicCoreName = "dc_cubes_historic"

In [37]:
hist_df = pd.DataFrame.from_dict(getData(historicCoreName))

In [38]:
allColumns = hist_df.columns.to_list()

In [39]:
url = "http://localhost:8983/solr/admin/cores?action=STATUS"
response = requests.get(url).json()
activeCores = response['status'].keys()

# if forecast core exists
if forecast_core_name in activeCores:
    print(forecast_core_name + " already exists")
    deleteSolrCore(forecast_core_name)
    # delete old data/predictions
    createSolrCore(forecast_core_name)
    # init schema
    initSchema(forecast_core_name, allColumns)
    #deleteCoreDocuments(forecast_core_name)
# else forecast core doesn't exist
else:
    print(forecast_core_name + " doesn't exist")
    # create an new forecast solr core
    createSolrCore(forecast_core_name)
    # init schema
    initSchema(forecast_core_name, allColumns)

dc_cubes_forecast doesn't exist
dc_cubes_forecast  created
dc_cubes_forecast  schema inited


In [40]:
# splits the data of each cube from the whole df in its own dataframe and takes only the last -history_steps
def splitInCubesFrames(df):
    unique_server_names = df.server.unique()
    splitted_frames = []
    for name in unique_server_names:
        new_df = df[df['server'] == name].copy()
        splitted_frames.append(new_df)
    return splitted_frames


In [41]:

measureInterval = 15 #min
hoursToPredict = 24 * 4
pred_horizon = int((60//measureInterval) * hoursToPredict) #(4*hours), timestep = 15min
days_history = 1
hours_history = int(24 * days_history)
n_history = int((60//measureInterval)*hours_history)

history_steps = n_history
forecast_steps = pred_horizon

In [42]:
timestamp = "timestamp"

In [43]:
hist_df.reset_index(inplace=True)

In [44]:
last_timestamp = hist_df.timestamp.max()
hist_df[timestamp] = pd.to_datetime(hist_df.timestamp)
# generate features/columns from the timestamp
hist_df["dayOfWeek"] = hist_df[timestamp].map(lambda x: x.dayofweek)
hist_df["isWeekend"] = hist_df.dayOfWeek.map(lambda x: 0 if (x < 5) else 1) # saturday.dayofweek = 5, monday=0
#hist_df["weekofyear"] = hist_df[timestamp].map(lambda x: x.weekofyear)
hist_df["hour"] = hist_df[timestamp].map(lambda x: x.hour)
hist_df["minute"]= hist_df[timestamp].map(lambda x: x.minute)
hist_df = hist_df.set_index('timestamp')

hist_df.index = pd.to_datetime(hist_df.index).sort_values()

In [45]:
from keras.models import load_model

# split cubes in own frames
cubes_frames = splitInCubesFrames(hist_df)

# load the trained model
modelDc0 = load_model('final_cnn_pblm2.h5')
modelDc1 = load_model('final_cnn_pblm1.h5')

Using TensorFlow backend.


In [46]:
def makePredictionFrame(modelDc0, modelDc1, cubes_frames, last_timestamp, predictionColumn = "cpuusage_ps"):
    prediction_frames = []
    for cube in cubes_frames:
        # transform pre prediction input

        # extracting information from the current server
        last_timestamp = cube.index[-1]
        server_name = cube['server'].iloc[0]
        cluster = cube['cluster'].iloc[0]
        dc = cube['dc'].iloc[0]
        perm = cube['perm'].iloc[0]
        instanz = cube['instanz'].iloc[0]
        verfahren = cube['verfahren'].iloc[0]
        service = cube['service'].iloc[0]

        print("DATACENTER: ", dc)
        if(int(dc) == 0):
            model = modelDc0
        else:
            model = modelDc1
            
        dropCols = ["id", "cluster",
                "_version_",
                "dc",
                "perm",
                "instanz",
                "verfahren",
                "service",
                "response",
                "minv",
                "maxv", 
                "avg",
                "var",
                "dev_upp",
                "dev_low",
                "perc90",
                "perc95",
                "perc99",
                "sum",
                "sum_of_squares",
                "server"]
        # Converting the index as date
#         pdb.set_trace()
        cube.index = pd.to_datetime(cube.index).sort_values()
        


        cube = cube.drop(dropCols,axis=1)
        dataset = cube
#         if dc == 0:
#             allCols = cube.columns
#             cube[allCols] = cube[allCols].apply(pd.to_numeric, errors="coerce")
#             dataset = cube.dropna(axis=1, how="all")
#             dataset["dummy1"] = 1
#             dataset["dummy2"] = 0
        y = dataset[predictionColumn].copy() # prediction column was pushed to count
        x = dataset.drop(columns=[predictionColumn, "count"])
#         print(x.shape)
        # Standardise
        allMetrics =  dataset.columns.tolist()
        print("X SHAPE", x.shape)

        # standardise
        scalerX = StandardScaler()
        scalerX.fit(x)
        x = scalerX.transform(x)
        scalerY = StandardScaler()
       # .reshape(-1, 1) # needed for standardScaler
        scalerY.fit(y.values.reshape(-1,1))
        y = scalerY.transform(y.values.reshape(-1,1))
        
        #PCA
#         pcaTransformer = PCA(n_components=63) # keep 95% variance
#         pcaTransformer.fit(x)
#         x = pcaTransformer.transform(x)

        #transformed_df = pd.DataFrame().from_records(x)
        #transformed_df[predictionColumn] = y
        numberOfFeatures = x.shape[1]
        # predict
        pred_input = x[x.shape[0]-history_steps:]
#         print("***mean, std", x.mean(), x.std())
#         print("************\n", pred_input)
#         pdb.set_trace()
        pred_input = pred_input.reshape(
            (1, history_steps, numberOfFeatures)) # alternative (1, pred_input.shape[0],pred_input.shape[1])
        prediction = model.predict(pred_input)
#         print("prediction: ", prediction, prediction.shape)
        #prediction = np.hstack((prediction, np.zeros((prediction.shape[0], numberOfFeatures-1), dtype=prediction.dtype)))
        
    #    print("pre inverse scaling", prediction)
        prediction = scalerY.inverse_transform(prediction)
#        print("post inverse scaling", prediction)
        # transform pre solr
#         prediction = prediction.reshape((forecast_steps, 1))
#         prediction = np.hstack((prediction, np.zeros(
#             (prediction.shape[0], 3), dtype=prediction.dtype)))
#         prediction = prediction = scaler.inverse_transform(prediction)
#         prediction = prediction[:, [0]]
#         int_prediction = prediction.astype(int, copy=True)
#         prediction = int_prediction

        prediction = prediction.reshape(prediction.shape[1])
        # make the dataframe
        next_timestamps = pd.date_range(
            start=last_timestamp, periods=forecast_steps+1, freq='15min',  closed='right')
        # create the prediction dataframe for the current server
        d = {'timestamp': next_timestamps, 'cluster': cluster, 'dc': dc,
             'perm': perm, 'instanz': instanz,  'verfahren': verfahren, 'service': service, 'response': 200}
        pred_df = pd.DataFrame(data=d)
        for metric in allMetrics:
            pred_df[metric] = -1
#         print("Length:", len(pred_df), "::::::", prediction.shape)
        pred_df['count'] = prediction
        pred_df[predictionColumn] = prediction
        pred_df['minv'] = 0
        pred_df['maxv'] = 0
        pred_df['avg'] = 0
        pred_df['var'] = 0
        pred_df['dev_upp'] = 0
        pred_df['dev_low'] = 0
        pred_df['perc90'] = 0
        pred_df['perc95'] = 0
        pred_df['perc99.9'] = 0
        pred_df['sum'] = 0
        pred_df['sum_of_squares'] = 0
        pred_df['server'] = server_name

        pred_df['timestamp'] = pred_df['timestamp'].dt.strftime(
            '%Y-%m-%dT%H:%M:00Z')
        prediction_frames.append(pred_df)
    print("Made predictions")
    return pd.concat(prediction_frames, ignore_index=True)



In [47]:
prediction_df = makePredictionFrame(modelDc0, modelDc1, cubes_frames, last_timestamp)

DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  0
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHAPE (2789, 176)
DATACENTER:  1
X SHA

In [48]:
def pushForecastData(row):
    global forecast_core_name
    global counter
    global allColumns
    # defining the api-endpoint
    url = "http://localhost:8983/solr/"+forecast_core_name+"/update/json/docs"
    # data to be sent to api
    data = {
                "timestamp": row["timestamp"],
                "cluster": row["cluster"],
                "dc": row["dc"],
                "perm": row["perm"],
                "instanz": row["instanz"],
                "verfahren": row["verfahren"],
                "service": row["verfahren"],
                "response": row["response"],
                "count": row["count"],
                "minv": row["minv"],
                "maxv": row["maxv"],
                "avg": row["avg"],
                "var": row["var"],
                "dev_upp": row["dev_upp"],
                "dev_low": row["dev_low"],
                "perc90": row["perc90"],
                "perc95": row["perc95"],
                "perc99": row["perc99.9"],
                "sum": row["sum"],
                "sum_of_squares": row["sum_of_squares"],
                "server": row["server"],
                "cpuusage_ps": row["cpuusage_ps"]
            }
    
    for col in allColumns:
        if col not in data:
            data[col] = -1
    data.pop("id", None)
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    session.get(url)
    headers = {'Content-type': 'application/json'}
    # sending post request
    session.post(url=url, data=json.dumps(data), headers=headers)
    counter += 1
    if (counter % 1000 == 0):
        print("Commiting... counter:", counter)
        requests.get("http://localhost:8983/solr/"+forecast_core_name+"/update?commit=true")

In [49]:
prediction_df.head()

,timestamp,cluster,dc,perm,instanz,verfahren,service,response,level_0,count,...,avg,var,dev_upp,dev_low,perc90,perc95,perc99.9,sum,sum_of_squares,server
0,2020-01-23T10:00:00Z,8,0,-1,2,-1,-1,200,-1,1.439919,...,0,0,0,0,0,0,0,0,0,PBZ08E00_PERM02_S02_OSB
1,2020-01-23T10:15:00Z,8,0,-1,2,-1,-1,200,-1,1.080806,...,0,0,0,0,0,0,0,0,0,PBZ08E00_PERM02_S02_OSB
2,2020-01-23T10:30:00Z,8,0,-1,2,-1,-1,200,-1,0.854486,...,0,0,0,0,0,0,0,0,0,PBZ08E00_PERM02_S02_OSB
3,2020-01-23T10:45:00Z,8,0,-1,2,-1,-1,200,-1,0.865069,...,0,0,0,0,0,0,0,0,0,PBZ08E00_PERM02_S02_OSB
4,2020-01-23T11:00:00Z,8,0,-1,2,-1,-1,200,-1,1.048858,...,0,0,0,0,0,0,0,0,0,PBZ08E00_PERM02_S02_OSB


In [50]:
prediction_df.apply(pushForecastData, axis=1)
print("Last Commit...")
requests.get("http://localhost:8983/solr/"+forecast_core_name+"/update?commit=true")

Commiting... counter: 90000
Commiting... counter: 91000
Commiting... counter: 92000
Commiting... counter: 93000
Commiting... counter: 94000
Commiting... counter: 95000
Commiting... counter: 96000
Commiting... counter: 97000
Commiting... counter: 98000
Commiting... counter: 99000
Commiting... counter: 100000
Commiting... counter: 101000
Last Commit...


<Response [500]>

In [51]:
merged_core_name = "dc_cubes_merged"

In [52]:
def mergeTwoCores(merged_core_name, core1, core2):
    url = "http://localhost:8983/solr/admin/cores?action=mergeindexes&core=" + merged_core_name + "&srcCore=" + core1 + "&srcCore=" + core2
    requests.get(url=url)
    # Commit to materialize changes
    requests.get("http://localhost:8983/solr/"+merged_core_name+"/update?commit=true")
    print(core1 + " and " +  core2 + " have been merged to " + merged_core_name)

In [53]:
# if merged core exists
if merged_core_name in activeCores:
    print(merged_core_name + " already exists")
    # delete old data/predictions
    deleteCoreDocuments(merged_core_name)
# else forecast core doesn't exist
else:
    print(merged_core_name + " doesn't exist")
    # create an new forecast solr core
    createSolrCore(merged_core_name)
    # init schema
    initSchema(merged_core_name, allColumns)

# merged historic and forecast core
mergeTwoCores(merged_core_name, historicCoreName, forecast_core_name)

dc_cubes_merged already exists
deleted old documents from dc_cubes_merged core
dc_cubes_historic and dc_cubes_forecast have been merged to dc_cubes_merged
